We solve the generalized eigenvalue problem

$$
A u = \lambda M u,
$$

where $A$ comes from $\int \nabla u \nabla v$, and $M$ from $\int u v$, on the space $H_0^1$

In [15]:
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random 
from random import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

# setup stiffness and mass matrix:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$
u_{n+1} = A^{-1} M u_n,
$$
and the Rayleigh quotient 
$$
\rho_n = \frac{(A u_n, u_n)}{(M u_n, u_n)}
$$
converges to the smallest eigen-value, with rate of convergence $\lambda_1 / \lambda_2$.

The preconditioned invese iteration (PINVIT), see Knyazef+Neymeyr, replaces $A^{-1}$ by an approximate inverse $C^{-1}$:

$$
\rho_n = \frac{(A u_n, u_n)}{(M u_n, u_n)} \\
w_n = C^{-1} (A u_n - \rho M u_n) \\
u_{n+1} = u_n + \alpha w_n
$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$
u_{n+1} = \operatorname{arg} \min_{v \in \{ u_n, w_n\}} \frac{(A v, v)}{(M v, v)} 
$$

In [16]:
# some help vectors
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

In [17]:
# random initial conditions, zero on boundary
freedofs = fes.FreeDofs()    
for i in range(len(u.vec)):
    u.vec[i] = random() if freedofs[i] else 0   

In [19]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    eval,evec = scipy.linalg.eigh(asmall, b=msmall)
    # print (eval / pi**2)
    # print (evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

2.0000000039078714
2.000000000938538
2.0000000002480904
2.0000000000864726
2.000000000048369
2.0000000000393356
2.0000000000371796
2.0000000000366653
2.000000000036539
2.0000000000365104
2.0000000000365006
2.000000000036501
2.000000000036501
2.0000000000365
2.0000000000364997
2.0000000000365
2.0000000000365006
2.0000000000364997
2.0000000000365006
2.0000000000364992
